In [8]:
from pyspark.sql import *
from pyspark.sql.functions import *
import logging

spark = SparkSession.builder.master('local').appName('app').getOrCreate()

class pipeline:
    """
    input: path of input data folder
    output: path of output processed data folder
    """
    def __init__(self,input,output):
        
        self.input = input
        self.output = output


    def business_transform(self):
        
        """
            Reads business dataset, cleans data and stores processed files in output destination
        """
        logging.info('Reading business dataset')
        df_b = spark.read.json(self.input+"yelp_academic_dataset_business.json")

        #take RestaurantsPriceRange2 value from attributes column :
        df_b = df_b.withColumn("price_range", col("attributes").getField("RestaurantsPriceRange2"))
        df_b = df_b.na.drop() #drop null values 
        df_b = df_b.filter(col("categories").contains("Restaurants")) #filter out non restaurant businesses
        df_b = df_b.drop("attributes") #drop attribute column
        logging.info('Saving business dataset')
        df_b.write.format("parquet").mode("overwrite").save(self.output + "business.parquet")

    def review_transform(self):
        
        """
            Reads review dataset, cleans data and stores processed files in output destination
        """
        logging.info('Reading Review Dataset')
        df_r = spark.read.json(self.input+"yelp_academic_dataset_review.json")
        df_r = df_r.na.drop()
        logging.info('Saving review dataset')
        df_r.write.format("parquet").mode("overwrite").save(self.output + "review.parquet")

    def user_transform(self):

        """
            Reads user dataset, cleans data and stores processed files in output destination
        """
        logging.info('Reading user dataset')
        df_u = spark.read.json(self.input+"yelp_academic_dataset_user.json")
        df_u = df_u.na.drop()
        logging.info('saving user dataset...')
        df_u.write.format("parquet").mode("overwrite").save(self.output + "user.parquet")
    
    def checkin_transform(self):

        """
            Reads checkin dataset, cleans data and stores processed files in output destination
        """

        logging.info("Reading checkin dataset")
        df_c = spark.read.json(self.input+"yelp_academic_dataset_checkin.json")
        df_c = df_c.withColumn("no_of_checkins",size(split(col("date"),",")))
        df_c = df_c.drop("date")
        logging.info('saving checkin dataset')
        df_c.write.format("parquet").mode("overwrite").save(self.output + "checkin.parquet")

    def tip_transform(self):
        
        """
            Reads tip dataset, cleans data and stores processed files in output destination
        """
        logging.info('Reading tip dataset')
        df_t = spark.read.json(self.input+"yelp_academic_dataset_tip.json")
        df_t = df_t.na.drop()
        logging.info('saving tip dataset...')
        df_t.write.format("parquet").mode("overwrite").save(self.output + "tip.parquet")






In [4]:
class Main:
    """
        Args:
            source: path of Azure blob storage containing datasets
            output: path of Azure blob storage to store processed parquet files after transforming+cleaning
    """
    def __init__(self):
        logging.basicConfig(filename='pipeline.log', level=logging.DEBUG)
        path = "/HdiNotebooks/"
        ETL_data = pipeline(path, path)
        ETL_data.business_transform()
        ETL_data.review_transform()
        ETL_data.user_transform()
        ETL_data.checkin_transform()
        ETL_data.tip_transform()


Main()

    

In [9]:
tipfile = spark.read.parquet('/HdiNotebooks/tip.parquet',header=True,inferSchema=True)

In [10]:
tipfile.write.saveAsTable("tip")

In [12]:
checkinfile = spark.read.parquet('/HdiNotebooks/checkin.parquet',header=True,inferSchema=True)
checkinfile.write.saveAsTable("checkin")

In [13]:
userfile = spark.read.parquet('/HdiNotebooks/user.parquet',header=True,inferSchema=True)
userfile.write.saveAsTable("user")

In [14]:
reviewfile = spark.read.parquet('/HdiNotebooks/review.parquet',header=True,inferSchema=True)
reviewfile.write.saveAsTable("review")

In [15]:
businessfile = spark.read.parquet('/HdiNotebooks/business.parquet',header=True,inferSchema=True)
businessfile.write.saveAsTable("business")

In [47]:
%%sql
SELECT * FROM user

Output()

In [46]:
%%sql
SELECT * FROM tip 
ORDER BY compliment_count DESC

Output()

In [26]:
%%sql
SELECT * FROM business

Output()

In [50]:
%%sql
SELECT * FROM review

Output()

In [51]:
%%sql
SELECT * FROM checkin 
ORDER BY no_of_checkins DESC

Output()

## Selecting top cities with highest number of highest rated restaurants

In [49]:
%%sql 

SELECT DISTINCT(city) FROM
(SELECT *
FROM business
JOIN checkin ON business.business_id = checkin.business_id
ORDER BY checkin.no_of_checkins DESC
LIMIT 10)

Output()

## ## Selecting top cities with highest number of lowest rated restaurants

In [52]:
%%sql 

SELECT DISTINCT(city) FROM
(SELECT *
FROM business
JOIN checkin ON business.business_id = checkin.business_id
ORDER BY checkin.no_of_checkins
LIMIT 10)

Output()

In [53]:
%%sql
SELECT * FROM business


Output()

## Selecting top 10 restaurants in New Orleans(city with highest rated restaurants) in terms of review counts and stars

In [66]:
%%sql
SELECT DISTINCT(name),stars FROM
(SELECT name,stars FROM business
WHERE city == "New Orleans"
ORDER BY review_count,stars DESC)
LIMIT 10


Output()